Перед отправкой почистить. Здесь куча хлама. =D

#### Оглавление:
1. [Объявление основных функций](#StimViz)
2. [Выбор файлов и выполнение преобразования](#Open_files)
3. [Объявление класса Point, методов ГА и функции получения проекции координат](#GA_preparation)
4. [Выполнение ГА](#GA)
5. Визуализация результата через StimViz


 ## <a name="StimViz"></a> **I.** Объявление основных функций

In [1]:
from dipy.io.streamline import load_trk
from dipy.io.image import load_nifti
import numpy as np
from dipy.align.imaffine import (transform_centers_of_mass,
                                 AffineMap,
                                 MutualInformationMetric,
                                 AffineRegistration)
from dipy.align.transforms import (TranslationTransform3D,
                                   RigidTransform3D,
                                   AffineTransform3D)
from dipy.tracking import streamline
import copy
import pickle
import math
import simnibs
from simnibs import sim_struct, run_simnibs
from simnibs.msh import (surface,
                        mesh_io,
                        read_msh)
import random

In [22]:
def my_deriv(t,line,h=1):
    if t==0:
        return (-3*line[t]+4*line[t+1]-line[t+2])/(2*h)
    elif t==len(line)-1:
        return (line[t-2]-4*line[t-1]+3*line[t])/(2*h)
    else:
        return (line[t+1]-line[t-1])/(2*h)

def load_elems(nodes,elems):

    import numpy as np

    elems = elems[elems[:,3]!= -1,:]
    # Computing rectangles
    tmp = nodes[elems-1,:]
    elems_min = tmp.min(axis=1)
    elems_max = tmp.max(axis=1)
    tmp = 0
    sizes = (elems_max-elems_min).max(axis=0)
    # It is the index to reduce the elements to check
    order_min = np.argsort(elems_min[:,0])
    return {"Nodes":nodes, "Elems":elems, "El_min":elems_min, "El_max":elems_max, "Sizes":sizes,"Order_min":order_min}

# a function to get e-field vector at a given position [x,y,z]
def get_field(ttt, point, my_field):
    best_ttt=get_ttrd(ttt,point)
    return np.dot(my_field[best_ttt['Nodes']-1].T, best_ttt['Weights'])

def get_ttrd(loaded_elems,point):
    import numpy as np
    # Just to use names I have used before
    nodes = loaded_elems["Nodes"]
    elems = loaded_elems["Elems"]
    elems_min = loaded_elems["El_min"]
    elems_max = loaded_elems["El_max"]
    sizes = loaded_elems["Sizes"]
    order_min = loaded_elems["Order_min"]
    
    # Binary search to reduce the iterating points from 4mln to around 200k.
    r = np.searchsorted(elems_min[:,0],point[0],side='left',sorter=order_min)
    l = np.searchsorted(elems_min[:,0],point[0] - sizes[0],side='right',sorter=order_min)
    # Projection the data to only these points
    e_max = elems_max[order_min[l:r]]
    e_min = elems_min[order_min[l:r]]
    
    # Checks which ttrds are possible to contain the point
    potential_ttrds = order_min[l:r][(point[0] <= e_max[:,0]) & (e_min[:,1]<= point[1]) & (point[1] <= e_max[:,1]) & (e_min[:,2]<= point[2]) & (point[2] <= e_max[:,2])]
    
    # It checks if the ttrd contains the point
    def check_ttrd(ttrd, point):
        coord = np.column_stack((ttrd[1,:]-ttrd[0,:],ttrd[2,:]-ttrd[0,:],ttrd[3,:]-ttrd[0,:]))
        coord = np.linalg.inv(coord).dot(point-ttrd[0,:])
        return coord.min() >= 0 and coord.sum() <= 1
    # It checks if the ttrd with num ttrdNum contains the point
    def check_ttrd_byNum(ttrdNum, point):
        ttrd = nodes[elems[ttrdNum]-1]
        return check_ttrd(ttrd,point)
    
    # Just takes all ttrds that contain points
    nodeIndices = elems[[x for x in potential_ttrds if check_ttrd_byNum(x,point)]][0]; 
    ns = nodes[nodeIndices-1]

    norms = np.sum((ns-point)**2,axis=-1)**0.5
    weights = 1/(norms+1e-10)
    weights = weights / weights.sum()
    
    return {"Nodes":nodeIndices,"Weights":weights}


# a function to calculate directional derivatives of the effective field at a given point [x,y,z]
def deriv_e_field(coordinates, e_field_nodes, LSD, ttt):
   
    step=0.05

    x1=coordinates[0]
    y1=coordinates[1]
    z1=coordinates[2]
    x0=coordinates[0]-step
    x2=coordinates[0]+step
    y0=coordinates[1]-step
    y2=coordinates[1]+step
    z0=coordinates[2]-step
    z2=coordinates[2]+step

    f_x0_y1_z1=np.dot(get_field(ttt,np.asarray([x0,y1,z1]), e_field_nodes), LSD)
    f_x2_y1_z1=np.dot(get_field(ttt,np.asarray([x2,y1,z1]), e_field_nodes), LSD)
    f_x1_y1_z1=np.dot(get_field(ttt,np.asarray([x1,y1,z1]), e_field_nodes), LSD)
    f_x1_y0_z1=np.dot(get_field(ttt,np.asarray([x1,y0,z1]), e_field_nodes), LSD)
    f_x1_y2_z1=np.dot(get_field(ttt,np.asarray([x1,y2,z1]), e_field_nodes), LSD)
    f_x1_y1_z0=np.dot(get_field(ttt,np.asarray([x1,y1,z0]), e_field_nodes), LSD)
    f_x1_y1_z2=np.dot(get_field(ttt,np.asarray([x1,y1,z2]), e_field_nodes), LSD)
    
    gradx=my_deriv(1,[f_x0_y1_z1,f_x1_y1_z1,f_x2_y1_z1], step)
    grady=my_deriv(1,[f_x1_y0_z1,f_x1_y1_z1,f_x1_y2_z1], step)
    gradz=my_deriv(1,[f_x1_y1_z0,f_x1_y1_z1,f_x1_y1_z2], step)
    
    return np.dot([gradx, grady, gradz], LSD)

def simulation(fnamehead, pathfem, pos_centre=[-74.296158, -10.213354, 28.307243], pos_ydir=[-74.217369, -37.293205, 20.05232], distance=4, current_change=1e6):
    # Initalize a session
    s = sim_struct.SESSION()
    # Name of head mesh
    s.fnamehead = fnamehead
    # Output folder
    s.pathfem = pathfem
    # Not to visualize results in gmsh when running simulations (else set to True)
    s.open_in_gmsh=False
    
    # Initialize a list of TMS simulations
    tmslist = s.add_tmslist()
    # Select coil. For full list of available coils, please see simnibs documentation
    tmslist.fnamecoil = 'Magstim_70mm_Fig8.nii.gz'
    
    # Initialize a coil position
    pos = tmslist.add_position()
    pos.centre = pos_centre # Place the coil over
    pos.pos_ydir = pos_ydir # Point the coil towards
    pos.distance = distance # Distance between coil and head
    pos.didt = current_change # Rate of change of current in the coil, in A/s.
    try:
        run_simnibs(s)
    except Exception as ex:
        print('Get ERROR by doing run_simnibs \n', ex)

def calc_TMS_effects(x, y, z, pos_ydir): # (change_TMS_effects)
    
    l1 = 2  # membrane space constant 2mm
    l2 = l1**2
    
    # x,y,z already projected on scalp and in right coordinate system.
    # that's why next line commented
    #position = [x-256/2, y-256/2, z-256/2]  # -256/2 because of a freesurfer RAS coordinate system
    current_out_dir = out_dir+str(x)+'_'+str(y)+'_'+str(z)
    try:
        simulation(mesh_path, current_out_dir, pos_centre=[x,y,z], pos_ydir=pos_ydir)
    except Exception as ex:
        print('EROR: simulation(..) in calc_TMS_effects \n', ex)

    mesh_file = current_out_dir+'/'+subject_name+'_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh'
    field_mesh = read_msh(mesh_file)
    field_as_nodedata = field_mesh.elmdata[0].as_nodedata()
    field_at_nodes = field_as_nodedata.value
    ttt = load_elems(field_mesh.nodes.node_coord, field_mesh.elm.node_number_list)

    effective_field = copy.deepcopy(new_streams_T1_array)

    for stream in range(len(new_streams_T1_array)):
        my_steam = copy.deepcopy(new_streams_T1_array[stream])
        print('starting _'+str(stream)+' out of '+str(len(new_streams_T1_array)))
        for t in range(len(my_steam[:, 0])):
            # -256/2 because of a freesurfer RAS coordinate system
            x = my_steam[t, 0]-256/2
            y = my_steam[t, 1]-256/2
            z = my_steam[t, 2]-256/2
            xyz = np.asarray([x, y, z])

            field_vector_xyz = get_field(ttt, xyz, field_at_nodes)

            effective_field[stream][t, 0] = l1*np.dot(field_vector_xyz, streams_array_derivative[stream][t, :])
            effective_field[stream][t, 1] = l2*deriv_e_field(xyz, field_at_nodes, streams_array_derivative[stream][t, :], ttt)
            effective_field[stream][t, 2] = effective_field[stream][t, 0] + effective_field[stream][t, 1]
            
    with open(current_out_dir+'/'+subject_name+'_effective_field.txt', 'wb') as f:
        pickle.dump(effective_field, f)
    f.close()
    # Поиск максимума среди всех точек:
    return max([max(effective_field[stream][:,2]) for stream in range(len(effective_field))])



## <a name="Open_files"></a>**II.** Выбор файлов и выполнение преобразования


In [3]:
# Ввод данных
tractography_file='Data/tracts.trk'

sft = load_trk(tractography_file, tractography_file)
streams = sft.streamlines
streams_array = np.asarray(streams)
print('imported tractography data:'+tractography_file)

fname_T1 = 'Data/T1fs_conform.nii.gz'#input("Please, specify the T1fs_conform image that has been generated during head meshing procedure. ")
data_T1, affine_T1 = load_nifti(fname_T1)

fname_FA = 'Data/dti_fa.nii' #input("Please, specify the FA image. ")
data_FA, affine_FA = load_nifti(fname_FA)
print('loaded T1fs_conform.nii and FA images')

mesh_path = 'Data/subject.msh' #input("Please, specify the head mesh file. ")

last_slach = max([i for i, ltr in enumerate(mesh_path) if ltr == '/'])+1
subject_name = mesh_path[last_slach:-4]


# specify the directory where you would like to save your simulation results
# input example:'/home/Example_data/Output'
out_dir = 'Output' #input("Please, specify the directory where you would like to save your simulation results. ")
out_dir = out_dir+'/simulation_at_pos_'

imported tractography data:Data/tracts.trk
loaded T1fs_conform.nii and FA images


In [4]:
# To avoid recalculating the code cell below each time.

with open('../new_streams_T1_array.txt', 'rb') as f:
    new_streams_T1_array = pickle.load(f)
f.close()

with open('../streams_array_derivative.txt', 'rb') as f:
    streams_array_derivative = pickle.load(f)
f.close()

In [ ]:
# Co-registration of T1fs_conform and FA images. Performed in 4 steps.
# Step 1. Calculation of the center of mass transform. Used later as starting transform.
c_of_mass = transform_centers_of_mass(data_T1, affine_T1, data_FA, affine_FA)
print('calculated c_of_mass transformation')

# Step 2. Calculation of a 3D translation transform. Used in the next step as starting transform.
nbins = 32
sampling_prop = None
metric = MutualInformationMetric(nbins, sampling_prop)
level_iters = [10000, 1000, 100]
sigmas = [3.0, 1.0, 0.0]
factors = [4, 2, 1]
affreg = AffineRegistration(metric=metric,
                            level_iters=level_iters,
                            sigmas=sigmas,
                            factors=factors)

transform = TranslationTransform3D()
params0 = None
starting_affine = c_of_mass.affine
translation = affreg.optimize(data_T1, data_FA, transform, params0,
                    affine_T1, affine_FA,
                    starting_affine=starting_affine)
print('calculated 3D translation transform')

# Step 3. Calculation of a Rigid 3D transform. Used in the next step as starting transform
transform = RigidTransform3D()
params0 = None
starting_affine = translation.affine
rigid = affreg.optimize(data_T1, data_FA, transform, params0, 
                        affine_T1, affine_FA,
                        starting_affine=starting_affine)
print('calculated Rigid 3D transform')

# Step 4. Calculation of an affine transform. Used for co-registration of T1 and FA images.
transform = AffineTransform3D()
params0 = None
starting_affine = rigid.affine
affine = affreg.optimize(data_T1, data_FA, transform, params0,
                         affine_T1, affine_FA,
                         starting_affine=starting_affine)

print('calculated Affine 3D transform')

identity = np.eye(4)

inv_affine_FA = np.linalg.inv(affine_FA)
inv_affine_T1 = np.linalg.inv(affine_T1)
inv_affine = np.linalg.inv(affine.affine)

# transforming streamlines to FA space
new_streams_FA = streamline.transform_streamlines(streams, inv_affine_FA)
new_streams_FA_array = np.asarray(new_streams_FA)

T1_to_FA = np.dot(inv_affine_FA, np.dot(affine.affine, affine_T1))
FA_to_T1 = np.linalg.inv(T1_to_FA)

# transforming streamlines from FA to T1 space
new_streams_T1 = streamline.transform_streamlines(new_streams_FA, FA_to_T1)
global new_streams_T1_array
new_streams_T1_array = np.asarray(new_streams_T1)

# calculating amline derivatives along the streamlines to get the local orientation of the streamlines
global streams_array_derivative
streams_array_derivative = copy.deepcopy(new_streams_T1_array)

print('calculating streamline derivatives')
for stream in range(len(new_streams_T1_array)):
    my_steam = new_streams_T1_array[stream]
    for t in range(len(my_steam[:, 0])):
        streams_array_derivative[stream][t, 0] = my_deriv(t, my_steam[:, 0])
        streams_array_derivative[stream][t, 1] = my_deriv(t, my_steam[:, 1])
        streams_array_derivative[stream][t, 2] = my_deriv(t, my_steam[:, 2])
        deriv_norm = np.linalg.norm(streams_array_derivative[stream][t, :])
        streams_array_derivative[stream][t, :] = streams_array_derivative[stream][t, :]/deriv_norm

-----------
## <a name="GA_preparation"></a> **III.** Объявление класса Point, методов ГА и функции получения проекции координат
class Point, get_parents_id(), crossover(parents_id), get_coords_from_angles(alfa, beta)

In [5]:
class Point:
    def __init__(self, angles):
        self.angles=angles
        self.coords=get_coords_from_angles(angles[0], angles[1]).tolist()
        
        
    def __lt__(self, other):
        return self.fitness > other.fitness
    
    def calc_fitness(self):
        try:
            x,y,z=self.coords
            path_to_possible_effective_field=out_dir+str(x)+'_'+str(y)+'_'+str(z)+'/'+subject_name+'_effective_field.txt'
            
            # if simulation on this coordinates somehow was done earlier
            if exists(path_to_possible_effective_field):
                with open(path_to_possible_effective_field, 'rb') as f:
                    effective_field = pickle.load(f)
                f.close()
                self.fitness=max([max(effective_field[stream][:,2]) for stream in range(len(effective_field))])
            
            # normal way
            else:
                rotation_angle=math.radians(self.angles[2])
                pos_ydir=get_coords_from_angles(self.angles[0] + math.sin(rotation_angle)*7, # just 7
                                                self.angles[1] + math.cos(rotation_angle)*7)
                #print('pos_ydir =', pos_ydir)
                self.fitness=calc_TMS_effects(*self.coords, pos_ydir)
        except Exception as ex:
            print('EROR_EROR_SO_SAD_EROR', self.coords)
            print(ex)
    
    def is_point_close_to_streams(self):
        return (self.coords[0]>=x_min and self.coords[0]<=x_max 
                and self.coords[1]>=y_min and self.coords[1]<=y_max 
                and self.coords[2]>=z_min and self.coords[2]<=z_max)

In [6]:
# Определение родительской пары
def get_parents_id():   
    
    def get_parent(fit_values):
        randnum=random.random()
        fitness_sum = fit_values.sum() #sum(fit_values)
        fit_values= fit_values/fitness_sum
        cdf=np.concatenate([np.array([0]), fit_values.cumsum()])
        for i in range(len(cdf)-1):
            if((randnum>cdf[i])&(randnum<=cdf[i+1])):
                return i
            
    #generation.sort()
    fit_values = np.array([x.fitness for x in  generation])
    parent_id_a= get_parent(fit_values)
    parent_id_b= get_parent(np.delete(fit_values, parent_id_a))
    if parent_id_b>=parent_id_a:
        parent_id_b+=1;
    return(parent_id_a, parent_id_b)

In [7]:
# Скрещивание
# Через арифметический кроссовер (по сути морфинг)
def crossover(parents_id): 
    angles_a = generation[parents_id[0]].angles
    angles_b = generation[parents_id[1]].angles
    w = random.random()
    
    # add 180 to min angle for better crossower of rotation (I hope)
    rotation_a = angles_a[2]
    rotation_b = angles_b[2]
    if abs(rotation_a-rotation_b)>180:
        if rotation_a<rotation_b:
            rotation_a+= 360
        else:
            rotation_b+=360
    
    angles_c1 = [w * angles_a[0] + (1-w) * angles_b[0],
                 w * angles_a[1] + (1-w) * angles_b[1],
                (w * rotation_a + (1-w) * rotation_b)%360]
    angles_c2 = [w * angles_b[0] + (1-w) * angles_a[0],
                 w * angles_b[1] + (1-w) * angles_a[1],
                (w * rotation_b + (1-w) * rotation_a)%360]
    return angles_c1, angles_c2

In [8]:
mesh_struct = mesh_io.read_msh(mesh_path)
scalp=surface.Surface(mesh_struct, [5,1005])

def get_coords_from_angles(alfa, beta):
    # (1) Преобразование в трехмерную систему координат. x,y,z ∈ [0,255]
    r=256/2
    a= math.radians(alfa)
    b= math.radians(beta)
    
    x= r * math.sin(a) * math.cos(b) +r
    y= r * math.cos(a) +r
    z= r * math.sin(a) * math.sin(b) +r
    
    coords=np.array((x,y,z))
    coords-=256/2
    
    # (2) Получение проекции координаты на голову
    proj_coords,_=scalp.projectPoint(coords)
    return proj_coords

---

In [9]:
# Ограничение в 25мм
x_min = np.min(new_streams_T1_array[0][:,0]) - 25 - 256/2
x_max = np.max(new_streams_T1_array[0][:,0]) + 25 - 256/2
y_min = np.min(new_streams_T1_array[0][:,1]) - 25 - 256/2
y_max = np.max(new_streams_T1_array[0][:,1]) + 25 - 256/2
z_min = np.min(new_streams_T1_array[0][:,2]) - 25 - 256/2
z_max = np.max(new_streams_T1_array[0][:,2]) + 25 - 256/2
print (x_min, x_max, y_min, y_max, z_min, z_max)

-58.35510653248957 6.999714416924746 -37.13407266408484 29.310656427633916 -31.340430584812097 93.1118822930465


## <a name="GA"></a>**IV.** Выполнение ГА:

----
## IV_1. Вычисления с использованием multiprocessing
----

### Многопроцессорность

In [10]:
import multiprocessing
from os.path import exists
import time

In [37]:
class Consumer(multiprocessing.Process):

    def __init__(self, task_queue, result_queue):
        multiprocessing.Process.__init__(self)
        self.task_queue = task_queue
        self.result_queue = result_queue

    def run(self):
        pname = self.name
            
        while True:
            point = self.task_queue.get()
            
            if point is None: # poison pill
                print('Exiting %s...' % pname)
                self.task_queue.task_done()
                break
                
            print('{0} processing task: {1}'.format(pname, point.coords))
            try:
                point.calc_fitness()
            except Exception as ex:
                print('=================ERRRROOOORRRRR RRRRRR!!!========= eror')
            
            self.task_queue.task_done()
            self.result_queue.put(point)

----
### Выполнение ГА
(I)
    1. Generate random list with angles (angles_list)
    2. Calculate projected coordinates and effect.
(II)
    3. Create new generation and calculate effect
    4. Repeat from II

In [28]:
# Settings
number_of_generations=20
number_of_children=6
individs_in_generation=15
mutation_start=2

In [30]:
# Only to visualize progress
# Also need to comment one line ("prg_bar.value+=1") code cell below
# if there is no need to visualize progress
from ipywidgets import IntProgress
from IPython.display import display
prg_bar = IntProgress(min = 0, max = number_of_generations) # Создаем прогрессбар
display(prg_bar)

IntProgress(value=0, max=20)

In [ ]:
print('\n\n\n\n\nEND\n\n\\n=========')

In [38]:
# GA with multiprocessing
generation=[]
points=[]
steps_with_unchanged_leader=0
leader=0

for step in range(number_of_generations):
    if step==0:
        # Create first generation
        while True:
            point=Point([random.uniform(0,180) for i in range(2)]+[random.uniform(0,360)])
            if point.is_point_close_to_streams():
                points.append(point)
            if (len(points)==10):
                break
        
    else:
        points.clear()
        # Generate children by crossover
        while len(points)<number_of_children:
            children_angles=crossover(get_parents_id())
            # uniqueness and distance tests
            for child_angles in children_angles:
                point= Point(child_angles)
                coords_in_generation=[x.coords for x in generation]
                if (point.is_point_close_to_streams() 
                    and point.coords not in [x.coords for x in points]
                    and point.coords not in coords_in_generation
                   ):
                    points.append(point)
        
    # Mutation
    if steps_with_unchanged_leader>=mutation_start:
        mutant_id=random.randint(3,len(generation)-1)
        angles_alfa_beta=generation[mutant_id].angles[:2]
        del(generation[mutant_id])
        while True:
            angles_a_b=angles_alfa_beta
            angles_a_b[random.getrandbits(1)]=random.uniform(0,180)
            mutant=Point(angles_a_b+[random.uniform(0,360)])
            if mutant.is_point_close_to_streams():
                points.append(mutant)
                break
        print('*mutation*')
    
    
    # multiprocessing calculation
    if __name__ == '__main__':
        tasks = multiprocessing.JoinableQueue()
        results = multiprocessing.Queue()
        # spawning consumers
        n_consumers = 2 #multiprocessing.cpu_count()
        print('Spawning %i consumers...' % n_consumers)
        consumers = [Consumer(tasks, results) for i in range(n_consumers)]

        for consumer in consumers:
            consumer.start()

        # enqueueing jobs
        for point in points:
            tasks.put(point)
        # poison pill
        for i in range(n_consumers):
            tasks.put(None)
        
#         for consumer in consumers:
#             consumer.join()
        
        tasks.join()
        
        for i in range(len(points)):
            generation.append(results.get())
            
    generation.sort()
    generation= generation[:individs_in_generation]
    
    if leader==generation[0].fitness:
        steps_with_unchanged_leader+=1
    else:
        steps_with_unchanged_leader=0
        leader=generation[0].fitness
    
    print('===== Generation {0} ====='.format(step))
    prg_bar.value+=1 # <--comment this line if there is no need to visualize progress
    for point in generation:
        print(point.coords, point.fitness)
    print('==========================')

*mutation*
Spawning 2 consumers...
Consumer-44 processing task: [-27.512163, 8.154368, 78.981918]
Consumer-43 processing task: [-17.00437, -2.506723, 84.094566]


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-27.512163_8.154368_78.981918
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-27.512163_8.154368_78.981918
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-17

INFO:simnibs:Result Files:
/home/roman/Документы/Notebooks/Output/simulation_at_pos_-17.00437_-2.506723_84.094566/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]INFO: SimNIBS finished running simulations
INFO:simnibs:SimNIBS finished running simulations
[ simnibs ]INFO: Simulation Result Meshes:
INFO:simnibs:Simulation Result Meshes:
[ simnibs ]INFO: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-17.00437_-2.506723_84.094566/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
INFO:simnibs:/home/roman/Документы/Notebooks/Output/simulation_at_pos_-17.00437_-2.506723_84.094566/subject_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh
[ simnibs ]INFO: =====================================
INFO:simnibs:=====================================
[ simnibs ]SUMMARY: 
subject_TMS_1-0001_Magstim_70mm_Fig8_nii
Gray Matter

Field Percentiles
-----------------
Top percentiles o

starting _0 out of 6
starting _0 out of 6
starting _1 out of 6
starting _1 out of 6
starting _2 out of 6
starting _2 out of 6
starting _3 out of 6
starting _3 out of 6
starting _4 out of 6
starting _4 out of 6
starting _5 out of 6
starting _5 out of 6
Consumer-44 processing task: [-25.016628, -0.034911, 81.143929]


[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-25.016628_-0.034911_81.143929
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-25.016628_-0.034911_81.143929


...

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-29.873165_-27.755114_78.032494
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-29.873165_-27.755114_78.032494
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-29.873165_-27.755114_78.032494
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-29.873165_-27.755114_78.032494
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnib

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
Exiting Consumer-68...
pup_pip
===== Generation 17 =====
[-32.893646, -3.504184, 77.740974] 6.61583234893137
[-31.7505, -0.622016, 78.127647] 6.570579399251757
[-32.41777, -1.860041, 77.918594] 6.546460515618461
[-32.41777, -1.860041, 77.918594] 6.546460515618461
[-35.016212, -5.99108, 76.663177] 6.524865169509143
[-31.007935, -0.00341, 78.420395] 6.4585503055031594
[-35.772923, -7.737505, 76.326111] 6.418459834736799
[-30.791065, 1.866657, 78.358505] 6.374983516178801
[-31.82266, -6.744162, 78.367149] 6.367142513485818
[-33.856903, -5.060771, 77.296944] 6.332433670719033
[-34.77264, -7.637337, 76.850266] 6.331782368013112
[-31.555162, 2.863444, 77.889458] 6.321106321673836
[-30.257067, -1.079734, 78.945457] 6.281959358939195
[-32.928921, -6.429111, 77.78582] 6.257748275800424
[-36.297291, -10.48024, 76.096504] 6.2547268989312785
*mutation*
Spawning 2 consumers.

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
INFO:simnibs:Head Mesh: /home/roman/Документы/Notebooks/Data/subject.msh
[ simnibs ]INFO: Subject Path: None
INFO:simnibs:Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-33.71072_25.414877_72.850548
INFO:simnibs:Simulation Folder: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-33.71072_25.414877_72.850548
[ simnibs ]INFO: Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-33.71072_25.414877_72.850548
INFO:simnibs:Running simulations in the directory: /home/roman/Документы/Notebooks/Output/simulation_at_pos_-33.71072_25.414877_72.850548
[ simnibs ]INFO: Running Poslist Number: 1
INFO:simnibs:Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
INFO:simnibs:Began 

starting _0 out of 6
starting _1 out of 6
starting _2 out of 6
starting _3 out of 6
starting _4 out of 6
starting _5 out of 6
Exiting Consumer-70...
pup_pip
===== Generation 18 =====
[-32.893646, -3.504184, 77.740974] 6.61583234893137
[-31.7505, -0.622016, 78.127647] 6.570579399251757
[-32.41777, -1.860041, 77.918594] 6.546460515618461
[-32.41777, -1.860041, 77.918594] 6.546460515618461
[-35.016212, -5.99108, 76.663177] 6.524865169509143
[-31.007935, -0.00341, 78.420395] 6.4585503055031594
[-35.772923, -7.737505, 76.326111] 6.418459834736799
[-30.791065, 1.866657, 78.358505] 6.374983516178801
[-31.82266, -6.744162, 78.367149] 6.367142513485818
[-33.856903, -5.060771, 77.296944] 6.332433670719033
[-34.77264, -7.637337, 76.850266] 6.331782368013112
[-31.555162, 2.863444, 77.889458] 6.321106321673836
[-30.257067, -1.079734, 78.945457] 6.281959358939195
[-32.928921, -6.429111, 77.78582] 6.257748275800424
[-31.903692, -5.037408, 78.301682] 6.2266396413485605


KeyboardInterrupt: 

In [18]:
# for some reasone code freeze on simulation
print(prg_bar.value)
print(generation[0].fitness, generation[0].coords)

2
5.815007778021416 [-27.122837, -3.928036, 80.598656]


----

In [39]:
for x in generation:
    print(x.fitness, x.angles, x.coords)

6.61583234893137 [90.84872540202352, 115.15918324816603, 173.80347774758226] [-32.893646, -3.504184, 77.740974]
6.570579399251757 [87.94455032838204, 113.87483858386439, 170.02386337914643] [-31.7505, -0.622016, 78.127647]
6.546460515618461 [89.47162963325431, 114.99961595257636, 166.70225728187512] [-32.41777, -1.860041, 77.918594]
6.546460515618461 [89.91434251590168, 115.29210094630763, 241.75276619761] [-32.41777, -1.860041, 77.918594]
6.524865169509143 [92.20999738164268, 116.00556809205979, 167.18463387571046] [-35.016212, -5.99108, 76.663177]
6.4585503055031594 [87.99995727099994, 113.34714632651588, 176.23863807628905] [-31.007935, -0.00341, 78.420395]
6.418459834736799 [93.37734320062906, 116.08884639072636, 172.14494815244993] [-35.772923, -7.737505, 76.326111]
6.374983516178801 [87.3526339200582, 113.02893429057343, 175.37637249101095] [-30.791065, 1.866657, 78.358505]
6.367142513485818 [92.46102556300839, 114.0295543696857, 188.96468787384777] [-31.82266, -6.744162, 78.3671

In [45]:
with open('../generation_10_steps.txt', 'wb') as f:
    pickle.dump(generation, f)
f.close()

In [43]:
with open('../generation.txt', 'rb') as f:
    generation = pickle.load(f)
f.close()

-------------------

In [35]:
individs_in_generation

15

In [40]:
steps_with_unchanged_leader

9

In [66]:
xx.is_point_close_to_streams()

False

In [65]:
xx=Point([0,0])

In [43]:
math.cos(math.radians(360))

1.0

In [44]:
get_coords_from_angles(100,100)

array([-10.739119, -23.084139,  87.241631])

Важные мелочи и не мелочи:
- Код не будет пересчитывать поле при условии, что найдет subject_effective_field в папке с координатами. Отсюда вытекает, что код не будет пересчитывать при другом наклоне катушки если уже существует рассчет по таким координатам.
- При выполнении спустя несколько генераций код глючит при выполнении симуляции

Что сделал:
+ add rotation. Угол 0->360. При скрещивании у потомков угол принимает значения в пределах меньшей дуги между углами родителей (если их расположить на окружноси)
+ add проверка на 25мм